In [27]:
import codecs
import sys
import numpy as np

In [3]:
path = '/Users/annanesterenko/Desktop/diplom_final'
sys.path.append(path)

## ARXIV

In [17]:
text_collection = f"{path}/data/ARXIV/test_arxiv_plain.txt"
triples_file=f"{path}/data/ARXIV/arxiv_triplets.txt"

docs = {}

with codecs.open(text_collection) as f:
    for line in f:
        id_, text = line.split(' ', maxsplit=1)
        doc = text[1:-1].split(' ')
        docs[id_] = doc

In [43]:
D1 = docs['1307.7598']
D2 = docs['math/0504051']
Q = docs['1209.0268']

In [92]:
k_1 = 2
b = 0.75

def count_word_freq(q, D):
    freq = 0
    for word in D:
        if word == q:
            freq += 1
            
    return freq

def idf(q, docs, count_docs):
    n = 0
    for doc in docs:
        if q in doc:
            n += 1
            
    return np.log((count_docs + 0.5) / (n + 0.5))

def bm25_score(Q, D, docs):   
    # avg len of docs in search demand
    sum_docs_len = 0
    count_docs = 0
    for doc in docs:
        sum_docs_len += len(doc)
        count_docs += 1
    avg_doc_len = sum_docs_len / count_docs
    
    score = 0
    for q_i in Q:
        freq = count_word_freq(q_i, D)
        score_q_i = freq * (k_1 + 1) / (freq + k_1 * (1 - b + b * len(D) / avg_doc_len))
        score_q_i *= idf(q_i, docs, count_docs)
        score += score_q_i

    return score

In [96]:
covered_triplets = 0
correct_triplets = 0

i = 0 

with codecs.open(triples_file) as fin:
    for line in fin:
        i += 1
        id1, id2, id3 = map(lambda x: x.split('/pdf/')[-1], line.split())
        if id1 in docs.keys() and id2 in docs.keys() and id3 in docs.keys():
            covered_triplets += 1
            query = docs[id1]
            good_req = docs[id2]
            bad_req = docs[id3]
            demand = [good_req, bad_req]
            correct_triplets += bm25_score(query, good_req, demand) > bm25_score(query, bad_req, demand)
        if i % 100 == 0:
            print(i, correct_triplets)

100 58
200 117
300 177
400 238
500 293
600 353
700 404
800 458
900 518
1000 566
1100 631
1200 693
1300 745
1400 804
1500 866
1600 928
1700 992
1800 1050
1900 1104
2000 1156
2100 1219
2200 1272
2300 1322
2400 1375
2500 1432
2600 1488
2700 1543
2800 1605
2900 1665
3000 1720
3100 1773
3200 1823
3300 1880
3400 1947
3500 2005
3600 2058
3700 2121
3800 2185
3900 2242
4000 2307
4100 2358
4200 2418
4300 2475
4400 2533
4500 2585
4600 2642
4700 2694
4800 2752
4900 2810
5000 2869
5100 2932
5200 2985
5300 3048
5400 3100
5500 3160
5600 3212
5700 3257
5800 3324
5900 3388
6000 3444
6100 3499
6200 3558
6300 3618
6400 3671
6500 3721
6600 3780
6700 3836
6800 3892
6900 3952
7000 3996
7100 4057
7200 4117
7300 4173
7400 4225
7500 4282
7600 4345
7700 4409
7800 4468
7900 4529
8000 4590
8100 4650
8200 4709
8300 4768
8400 4821
8500 4870
8600 4924
8700 4980
8800 5038
8900 5095
9000 5153
9100 5207
9200 5273
9300 5335
9400 5396
9500 5451
9600 5512
9700 5570
9800 5627
9900 5684
10000 5745
10100 5813
10200 5865
1030

In [97]:
print(1.0 * correct_triplets / covered_triplets)

0.7297041569419037
